# Section 1 : Data Ingestion and Sqlite setup

In [ ]:
import os
import sqlite3
import pandas as pd
from sqlalchemy import create_engine


# Path to our DB
db_path = "/home/Davcote/Desktop/HPV-Awareness-Impact-Analysis/data/database/HPV.db"
engine = create_engine(f"sqlite:///{db_path}")

# Connect
conn = sqlite3.connect(db_path)

def run_sql(query):
    """Run SQL query and return results as a pandas DataFrame"""
    return pd.read_sql_query(query, conn)

def insert_processed(df: pd.DataFrame, name: str, engine=engine):
    """
    Inserts a processed DataFrame into the database.
    
    Parameters:
    - df (pd.DataFrame): The DataFrame to insert
    - name (str): The table name (will be prefixed with 'processed_')
    - engine: SQLAlchemy engine (default = HPV.db engine)

    Usage : insert_processed(df_name, "Name_of_table_you_want_to_create")
    """
    table_name = f"processed_{name}"
    try:
        df.to_sql(table_name, con=engine, if_exists="replace", index=False)
        print(f"✅ Inserted into table: {table_name} ({len(df)} rows)")
    except Exception as e:
        print(f"❌ Error inserting {table_name}: {e}")



def insert_file_to_db(file_path: str, engine=engine):
    """
    Inserts a CSV or Excel file into the SQLite DB.
    If the table exists, it will be replaced.

    Parameters:
    - file_path (str): Path to the file (.csv, .xls, .xlsx)
    - engine: SQLAlchemy engine

    Usage : insert_file_to_db("/fullpath/to/your/file.csv or xlsx or xls")
    """
    try:
        if not os.path.exists(file_path):
            print(f"❌ File not found: {file_path}")
            return

        # Derive table name from filename
        table_name = os.path.splitext(os.path.basename(file_path))[0]

        # Load file
        if file_path.lower().endswith(".csv"):
            df = pd.read_csv(file_path)
            df.to_sql(table_name, con=engine, if_exists="replace", index=False)
            print(f"✅ Inserted CSV into table: {table_name} ({len(df)} rows)")

        elif file_path.lower().endswith((".xls", ".xlsx")):
            xls = pd.read_excel(file_path, sheet_name=None)  # dict of DataFrames
            for sheet_name, df in xls.items():
                sheet_table = f"{table_name}_{sheet_name}"
                df.to_sql(sheet_table, con=engine, if_exists="replace", index=False)
                print(f"✅ Inserted Excel sheet '{sheet_name}' into table: {sheet_table} ({len(df)} rows)")
        else:
            print(f"❌ Unsupported file type: {file_path}")

    except Exception as e:
        print(f"❌ Error inserting {file_path}: {e}")



Replacing raw `summary_data.xlsx` dataset with processed `summary_data.xlsx` dataset, because
- raw summary_data sheets contained "comments" with column names, so we removed them using Excel
- there were `spaces` with names of sheets, so we manually removed them using Excel

In [ ]:
insert_file_to_db("/home/Davcote/Desktop/HPV-Awareness-Impact-Analysis/data/processed_data/summary_data.xx")

✅ Inserted Excel sheet 'demographic' into table: summary_data_demographic (58 rows)
✅ Inserted Excel sheet 'pretest' into table: summary_data_pretest (61 rows)
✅ Inserted Excel sheet 'post_test' into table: summary_data_post_test (61 rows)


# Listing all Tables present inside our Database

In [ ]:
# List all tables in the DB to verify
tables = run_sql("SELECT name FROM sqlite_master WHERE type='table';")
display(tables) #can use print(tables) as well for non-notebook environment

,name
0,raw_data_PRE_TEST
1,raw_data_HPV-KS
2,raw_data_demographic_variables
3,raw_data_1_DEMO_coded
4,raw_data_1_HPV_CODED
5,raw_data_POST_TEST
6,raw_data_DEMO_2
7,raw_data_coded_demo_2
8,raw_data_HPV_KS_2
9,raw_data_coded_hpv_2


In [33]:
# Assigning summary_data_demographic table to a dataframe and displaying first 5 rows
df_demo = run_sql("SELECT * FROM 'summary_data_demographic' LIMIT 5;")
display(df_demo)


,Sno,1,2,3,4,5,6,7,8
0,1,1,1,0,1,2,0,0,2
1,2,0,0,2,1,1,2,3,0
2,3,0,1,0,1,1,2,2,0
3,4,1,1,2,1,2,0,0,3
4,5,0,0,1,1,2,2,2,0


In [ ]:
# Assigning summary_data_pretest table to a dataframe and displaying first 5 rows
df_pre = run_sql("SELECT * FROM 'summary_data_pretest' LIMIT 5;")
display(df_pre)

,Sno,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,Total points
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,22.0
1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,17.0
3,4.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0
4,5.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,15.0


In [ ]:
# Assigning summary_data_posttest table to a dataframe and displaying first 5 rows
df_post = run_sql("SELECT * FROM 'summary_data_post_test' LIMIT 5;")
display(df_post)


,Sno,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,Total
0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,19.0
1,2.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.0
2,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,23.0
3,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,24.0
4,5.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,24.0


In [35]:
# Displaying info of demographic dataframe
df_demo.info()
df_pre.info()
df_post.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Sno     5 non-null      int64
 1   1       5 non-null      int64
 2   2       5 non-null      int64
 3   3       5 non-null      int64
 4   4       5 non-null      int64
 5   5       5 non-null      int64
 6   6       5 non-null      int64
 7   7       5 non-null      int64
 8   8       5 non-null      int64
dtypes: int64(9)
memory usage: 492.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 35 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sno           5 non-null      float64
 1   1             5 non-null      float64
 2   2             5 non-null      float64
 3   3             5 non-null      float64
 4   4             5 non-null      float64
 5   5             5 non-null      float64
 6   6             5 non-nu

Conclusion : Datatype of each table is correctly assigned (float64)

# Section 2 : Data Cleaning and Preprocessing

In [36]:
# 3. Data Cleaning and Merging
# Handle missing values, correct data types.
# Merge the separate dataframes into one comprehensive dataframe.
# Example: df_full = pd.merge(df_demo, df_pre_post, on='Participant_ID')